In [ ]:
pip install tfds

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from tensorflow.keras import layers, models
from tensorflow.keras.applications import (
    ResNet50,
    EfficientNetB0,
    resnet50,
    efficientnet
)
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the tf_flowers Dataset
# ---------------------------------------------------
# Create your own train/test split: 80% / 20%
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in tf_flowers
num_classes = info.features['label'].num_classes
nums = num_classes  # keep both variables as you did before

print(f"Number of classes: {num_classes}")

# ---------------------------------------------------
# 2. Convert to NumPy arrays (Optionally Resize)
# ---------------------------------------------------
IMG_SIZE = 128

x_train_list = []
y_train_list = []

for img, label in dataset_train:
    # Resize to reduce memory usage
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_train_list.append(img.numpy())
    y_train_list.append(label.numpy())

x_test_list = []
y_test_list = []

for img, label in dataset_test:
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    x_test_list.append(img.numpy())
    y_test_list.append(label.numpy())

# Convert lists to NumPy arrays
x_train = np.stack(x_train_list).astype("float32")
y_train = np.array(y_train_list)
x_test  = np.stack(x_test_list).astype("float32")
y_test  = np.array(y_test_list)

# ---------------------------------------------------
# 3. Normalize & One-Hot Encode Labels
# ---------------------------------------------------
# Normalize pixel values to [0, 1]
x_train /= 255.0
x_test  /= 255.0

# Convert integer labels to one-hot vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test  shape: {x_test.shape}")
print(f"y_test  shape: {y_test.shape}")
print(f"num_classes: {num_classes}, nums: {nums}")


Number of classes: 5
x_train shape: (2936, 128, 128, 3)
y_train shape: (2936, 5)
x_test  shape: (734, 128, 128, 3)
y_test  shape: (734, 5)
num_classes: 5, nums: 5


In [8]:

# We'll define two separate pipelines:
#   - One for ResNet50
#   - One for EfficientNetB0

# ------------------------------
# 2. tf.data Pipeline for ResNet
# ------------------------------
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    # Resize and apply ResNet-specific preprocessing
    image = tf.image.resize(image, (224, 224))
    image = resnet50.preprocess_input(image)
    return image, label

# Create training dataset
train_ds_resnet = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds_resnet = train_ds_resnet.shuffle(buffer_size=50000) \
    .map(preprocess_resnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

# Create validation (test) dataset
val_ds_resnet = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_ds_resnet = val_ds_resnet.map(preprocess_resnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)


# ------------------------------------------
# 4. Define Baseline Models to Fine-Tune
# ------------------------------------------
def create_resnet50_finetune(input_shape, num_classes):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    # Freeze base model
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# ----------------------------------------------
# 5. Fine-Tune ResNet50 on CIFAR-100
# ----------------------------------------------
resnet_model = create_resnet50_finetune((224, 224, 3), num_classes)
resnet_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning ResNet50 on CIFAR-100 ---")
history_resnet = resnet_model.fit(
    train_ds_resnet,
    epochs=3,
    validation_data=val_ds_resnet,
    verbose=1
)
resnet_loss, resnet_acc = resnet_model.evaluate(val_ds_resnet, verbose=0)
print(f"ResNet50 - CIFAR-100 Accuracy: {resnet_acc:.4f}")



--- Fine-Tuning ResNet50 on CIFAR-100 ---
Epoch 1/3
92/92 [==============================] - 48s 95ms/step - loss: 1.6381 - accuracy: 0.3004 - val_loss: 1.5210 - val_accuracy: 0.3351
Epoch 2/3
92/92 [==============================] - 6s 68ms/step - loss: 1.5180 - accuracy: 0.3532 - val_loss: 1.5176 - val_accuracy: 0.3229
Epoch 3/3
92/92 [==============================] - 7s 73ms/step - loss: 1.5082 - accuracy: 0.3420 - val_loss: 1.4880 - val_accuracy: 0.3665
ResNet50 - CIFAR-100 Accuracy: 0.3665


In [12]:
# ------------------------------
# 3. tf.data Pipeline for EfficientNet
# ------------------------------
def preprocess_efficientnet(image, label):
    # Resize and apply EfficientNet-specific preprocessing
    image = tf.image.resize(image, (224, 224))
    image = efficientnet.preprocess_input(image)
    return image, label

train_ds_eff = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds_eff = train_ds_eff.shuffle(buffer_size=50000) \
    .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)

val_ds_eff = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_ds_eff = val_ds_eff.map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(AUTOTUNE)



def create_efficientnet_finetune(input_shape, num_classes):
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    # Freeze base model
    base_model.trainable = False

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model



# ----------------------------------------------
# 6. Fine-Tune EfficientNetB0 on CIFAR-100
# ----------------------------------------------
eff_model = create_efficientnet_finetune((224, 224, 3), num_classes)
eff_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning EfficientNetB0 on CIFAR-100 ---")
history_eff = eff_model.fit(
    train_ds_eff,
    epochs=3,
    validation_data=val_ds_eff,
    verbose=1
)
eff_loss, eff_acc = eff_model.evaluate(val_ds_eff, verbose=0)
print(f"EfficientNetB0 - CIFAR-100 Accuracy: {eff_acc:.4f}")



--- Fine-Tuning EfficientNetB0 on CIFAR-100 ---
Epoch 1/3
1563/1563 [==============================] - 109s 67ms/step - loss: 0.5143 - accuracy: 0.8315 - val_loss: 0.3506 - val_accuracy: 0.8838
Epoch 2/3
1563/1563 [==============================] - 100s 64ms/step - loss: 0.2873 - accuracy: 0.9016 - val_loss: 0.3390 - val_accuracy: 0.8914
Epoch 3/3
1563/1563 [==============================] - 96s 62ms/step - loss: 0.2076 - accuracy: 0.9289 - val_loss: 0.3440 - val_accuracy: 0.8962
EfficientNetB0 - CIFAR-100 Accuracy: 0.8962


In [ ]:
################
################
#   GATING     #
################
################

In [1]:
import tensorflow as tf

class DynamicChannelGate(tf.keras.layers.Layer):
    """
    A learnable gating mechanism to dynamically prune (or re-expand) channels.
    Each channel has a gating parameter in [0, 1], learned via a sigmoid.
    """
    def __init__(self, num_channels, name=None):
        super().__init__(name=name)
        
        self.gate_params = tf.Variable(
            initial_value=tf.ones((num_channels,), dtype=tf.float32),
            trainable=True,
            name="gate_params"
        )

    def call(self, inputs, training=None):
        
        gate = tf.sigmoid(self.gate_params)
        
        # Reshape to broadcast across (batch, H, W, C)
        gate = tf.reshape(gate, (1, 1, 1, -1))
        
        return inputs * gate

    
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

def create_efficientnet_dynamic_gating(input_shape, num_classes):

    # 1. Load base EfficientNet (feature extractor)

    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    base_model.trainable = True
    x = base_model.output  

    # 3. Insert the DynamicChannelGate
    num_channels = x.shape[-1]
    gating_layer = DynamicChannelGate(num_channels=num_channels, name="dynamic_gating")
    x = gating_layer(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)  
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EFN_DynamicGating")
    return model


import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Create the dynamic-gating EfficientNet
model = create_efficientnet_dynamic_gating((224, 224, 3), num_classes=nums)

# Compile the model (tweak hyperparameters as needed)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning Dynamic-Gating EfficientNetB0 on CIFAR-100 ---")
history = model.fit(
    train_ds_eff,             # from your existing pipeline
    epochs=3,                 # increase for better results
    validation_data=val_ds_eff,
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(val_ds_eff, verbose=0)
print(f"Final Accuracy with Dynamic Gating: {acc:.4f}")



--- Initializing Dynamic Gating Model Setup ---

--- Loading and Preprocessing CIFAR-100 Dataset ---


ResourceExhaustedError: OOM when allocating tensor with shape[50000,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear]

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, resnet50
from tensorflow.keras.optimizers import Adam

###############################################################################
# 1. Data: CIFAR-10 loading & preprocessing for ResNet50
###############################################################################
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
num_classes = 10

x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test,  num_classes)

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (224, 224))
    # Use ResNet50's preprocessing
    image = resnet50.preprocess_input(image)
    return image, label

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32

train_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(50000)
    .batch(batch_size)
    .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(batch_size)
    .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

###############################################################################
# 2. DynamicChannelGate Layer
###############################################################################
class DynamicChannelGate(tf.keras.layers.Layer):
    """
    A learnable gating mechanism to dynamically prune (or re-expand) channels.
    Each channel has a gating parameter in [0, 1], learned via a sigmoid.
    """
    def __init__(self, num_channels, name=None):
        super().__init__(name=name)
        self.gate_params = tf.Variable(
            initial_value=tf.ones((num_channels,), dtype=tf.float32),
            trainable=True,
            name="gate_params"
        )

    def call(self, inputs, training=None):
        gate = tf.sigmoid(self.gate_params)          # shape (C,)
        gate = tf.reshape(gate, (1, 1, 1, -1))       # broadcast to (1, 1, 1, C)
        return inputs * gate

###############################################################################
# 3. Create a ResNet50 model with Dynamic Gating
###############################################################################
def create_resnet_dynamic_gating(input_shape, num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    # Optionally unfreeze the base for deeper fine-tuning
    base_model.trainable = True

    x = base_model.output
    num_channels = x.shape[-1]  # Typically 2048 for ResNet50
    gating_layer = DynamicChannelGate(num_channels=num_channels, name="dynamic_gating")
    x = gating_layer(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet_DynamicGating")
    return model

###############################################################################
# 4. Build, Train, and Evaluate the Model
###############################################################################
model = create_resnet_dynamic_gating((224, 224, 3), num_classes)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print("\n--- Fine-Tuning Dynamic-Gating ResNet50 on CIFAR-10 ---")
history = model.fit(
    train_ds,
    epochs=3,
    validation_data=test_ds,
    verbose=1
)

loss, acc = model.evaluate(test_ds, verbose=0)
print(f"\nFinal Accuracy with Dynamic Gating: {acc:.4f}")



--- Fine-Tuning Dynamic-Gating ResNet50 on CIFAR-10 ---
Epoch 1/3
1563/1563 [==============================] - 279s 174ms/step - loss: 0.3492 - accuracy: 0.8820 - val_loss: 0.2802 - val_accuracy: 0.9078
Epoch 2/3
1563/1563 [==============================] - 273s 174ms/step - loss: 0.1462 - accuracy: 0.9512 - val_loss: 0.2325 - val_accuracy: 0.9230
Epoch 3/3
1563/1563 [==============================] - 271s 173ms/step - loss: 0.0976 - accuracy: 0.9669 - val_loss: 0.2217 - val_accuracy: 0.9304

Final Accuracy with Dynamic Gating: 0.9304


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import numpy as np

# ---------------------------------------------------
# 1. Load the CIFAR-100 Dataset
# ---------------------------------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(len(np.unique(y_test)))

AttributeError: module 'keras.api._v2.keras.datasets' has no attribute 'food101'

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

print("\n--- Initializing Dynamic Gating Model Setup ---")  # Pre-training print statement

class DynamicChannelGate(tf.keras.layers.Layer):
    """
    A learnable gating mechanism to dynamically prune (or re-expand) channels.
    Each channel has a gating parameter in [0, 1], learned via a sigmoid.
    """
    def __init__(self, num_channels, name=None):
        super().__init__(name=name)
        
        self.gate_params = tf.Variable(
            initial_value=tf.ones((num_channels,), dtype=tf.float32),
            trainable=True,
            name="gate_params"
        )

    def call(self, inputs, training=None):
        print("DynamicChannelGate layer called.")  # Log when the layer is called
        tf.print("Shape of inputs:", tf.shape(inputs))  # Use tf.print for TensorFlow tensors
        
        gate = tf.sigmoid(self.gate_params)
        tf.print("Gate parameters (sigmoid applied):", gate)  # Log gate parameter values
        
        # Reshape to broadcast across (batch, H, W, C)
        gate = tf.reshape(gate, (1, 1, 1, -1))
        return inputs * gate


def create_efficientnet_dynamic_gating(input_shape, num_classes):
    print("Creating EfficientNet model with Dynamic Gating...")  # Log model creation
    
    # Load base EfficientNet (feature extractor)
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    base_model.trainable = True
    x = base_model.output  

    # Insert the DynamicChannelGate
    num_channels = x.shape[-1]
    gating_layer = DynamicChannelGate(num_channels=num_channels, name="dynamic_gating")
    x = gating_layer(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)  
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EFN_DynamicGating")
    print("Model creation completed.")  # Log model creation completion
    return model


def preprocess_data(image, label):
    """
    Resizes and normalizes the image and retains the label.
    """
    image = tf.image.resize(image, (224, 224)) / 255.0  # Resize and normalize
    return image, label


# Load CIFAR-100 dataset
print("\n--- Loading CIFAR-100 Dataset ---")
(x_train, y_train), (x_val, y_val) = cifar100.load_data()

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=100)
y_val = to_categorical(y_val, num_classes=100)

# Create TensorFlow datasets with batched preprocessing
batch_size = 32  # Small batch size to prevent OOM
train_ds_eff = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds_eff = train_ds_eff.batch(batch_size).map(preprocess_data).prefetch(tf.data.AUTOTUNE)

val_ds_eff = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_ds_eff = val_ds_eff.batch(batch_size).map(preprocess_data).prefetch(tf.data.AUTOTUNE)

# Create the dynamic-gating EfficientNet
nums = 100  # CIFAR-100 has 100 classes
model = create_efficientnet_dynamic_gating((224, 224, 3), num_classes=nums)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Fine-Tuning Dynamic-Gating EfficientNetB0 on CIFAR-100 ---")  # Log training start
history = model.fit(
    train_ds_eff,             
    epochs=3,                 
    validation_data=val_ds_eff,
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(val_ds_eff, verbose=0)
print(f"Final Accuracy with Dynamic Gating: {acc:.4f}")



--- Initializing Dynamic Gating Model Setup ---

--- Loading CIFAR-100 Dataset ---
Creating EfficientNet model with Dynamic Gating...
DynamicChannelGate layer called.
Model creation completed.

--- Fine-Tuning Dynamic-Gating EfficientNetB0 on CIFAR-100 ---
Epoch 1/3
DynamicChannelGate layer called.
DynamicChannelGate layer called.
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731058538 0.731058538 0.731058538 ... 0.731058538 0.731058538 0.731058538]
   1/1563 [..............................] - ETA: 18:13:37 - loss: 4.7461 - accuracy: 0.0000e+00Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731075525 0.731077969 0.731039286 ... 0.731040895 0.731039107 0.731039107]
   2/1563 [..............................] - ETA: 11:07 - loss: 4.7383 - accuracy: 0.0000e+00   Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731090844 0.731080413 0.731025577 ... 0.731025875 0.73104316 0.731019795]
   3/1563 [..............................] 

  34/1563 [..............................] - ETA: 11:46 - loss: 4.6028 - accuracy: 0.0147Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731317878 0.731337249 0.731147647 ... 0.730692 0.731001496 0.730938137]
  35/1563 [..............................] - ETA: 11:45 - loss: 4.6048 - accuracy: 0.0152Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731321752 0.731331885 0.731155813 ... 0.730686843 0.730995476 0.730937898]
  36/1563 [..............................] - ETA: 11:46 - loss: 4.6028 - accuracy: 0.0148Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731325 0.731325269 0.731163859 ... 0.730681241 0.73099 0.730939329]
  37/1563 [..............................] - ETA: 11:45 - loss: 4.6014 - accuracy: 0.0152Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731326103 0.731321454 0.731172502 ... 0.730676055 0.730981648 0.730938137]
  38/1563 [..............................] - ETA: 11:45 - loss: 4.6003 - accur

Gate parameters (sigmoid applied): [0.73126632 0.731184781 0.731401622 ... 0.730670214 0.730978727 0.731120169]
  70/1563 [>.............................] - ETA: 11:39 - loss: 4.5602 - accuracy: 0.0246Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731264293 0.731173575 0.731411934 ... 0.73066932 0.730980277 0.731133342]
  71/1563 [>.............................] - ETA: 11:39 - loss: 4.5604 - accuracy: 0.0251Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731261492 0.731163263 0.731424689 ... 0.730665 0.730981529 0.731137216]
  72/1563 [>.............................] - ETA: 11:38 - loss: 4.5594 - accuracy: 0.0252Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731261432 0.731155813 0.731435537 ... 0.730661094 0.73098129 0.731139541]
  73/1563 [>.............................] - ETA: 11:38 - loss: 4.5574 - accuracy: 0.0253Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731260955 0.731146455 0.731441736 ..

 105/1563 [=>............................] - ETA: 11:30 - loss: 4.5014 - accuracy: 0.0363Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731347 0.731005728 0.731511831 ... 0.730479538 0.731000841 0.731316149]
 106/1563 [=>............................] - ETA: 11:30 - loss: 4.4998 - accuracy: 0.0369Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731347859 0.73100841 0.731510222 ... 0.730471551 0.730998874 0.731328189]
 107/1563 [=>............................] - ETA: 11:30 - loss: 4.4982 - accuracy: 0.0368Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731349528 0.731011569 0.731503189 ... 0.73046273 0.730998337 0.731343567]
 108/1563 [=>............................] - ETA: 11:29 - loss: 4.4967 - accuracy: 0.0367Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731351 0.731010914 0.731494784 ... 0.730456293 0.731000125 0.731360316]
 109/1563 [=>............................] - ETA: 11:29 - loss: 4.4951 - acc

 140/1563 [=>............................] - ETA: 11:18 - loss: 4.4400 - accuracy: 0.0458Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731490374 0.730964 0.731539607 ... 0.730263114 0.730766654 0.731663048]
 141/1563 [=>............................] - ETA: 11:18 - loss: 4.4392 - accuracy: 0.0461Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731495738 0.730965495 0.731545806 ... 0.730258226 0.730762 0.731673896]
 142/1563 [=>............................] - ETA: 11:18 - loss: 4.4368 - accuracy: 0.0464Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.731504858 0.730967 0.731549263 ... 0.730251133 0.730759919 0.731688797]
 143/1563 [=>............................] - ETA: 11:17 - loss: 4.4362 - accuracy: 0.0465Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.73151058 0.730968535 0.731552899 ... 0.730243325 0.730757415 0.731703699]
 144/1563 [=>............................] - ETA: 11:17 - loss: 4.4344 - accur

...

Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]

Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]

Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]

Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]

Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]
Shape of inputs: [32 7 7 1280]
Gate parameters (sigmoid applied): [0.748230338 0.741971433 0.741517484 ... 0.736951947 0.73939389 0.744521558]